In [ ]:
spark

In [ ]:
# set max columns, rows, column width in pandas so doesn't truncate
import pandas as pd
pd.set_option('display.max_colwidth',250) # or -1
pd.set_option('display.max_columns', None) # or 500
pd.set_option('display.max_rows', None) # or 500

# sets the cell width to 100% respective to the screen size
from IPython.core.display import display, HTML
from pyspark.sql.functions import when, col
from pyspark.sql.functions import avg
display(HTML("<style>.container { width:92% !important; }</style>"))
from pyspark.sql.functions import col, sum as spark_sum
import pyspark.sql.functions as F
from pyspark.sql.functions import col, monotonically_increasing_id
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from pyspark.sql.functions import col, log
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



### Call in data and obtain caliper radius

In [ ]:
spark.sql('use CUA_db')

In [ ]:
#Call in saved dataframe with propensity scores
cua_non= spark.sql("""
    SELECT personid, CUA_ANY, prob1 AS prob
    FROM cua_non_age_PSM_update
""")
cua_non

In [ ]:
#Calculate and use logit of PS for estimating the caliper width in KNN matching
cua_non_logit = cua_non.withColumn('logit_prob', log(col('prob') / (1 - col('prob'))))

In [ ]:
logit_std_dev = cua_non_logit.agg(F.stddev('logit_prob')).collect()[0][0]

# Set the caliper radius to be 0.2 times the standard deviation of the logit
caliper_radius = 0.2 * logit_std_dev

print("Caliper Radius from all data:",caliper_radius)
#Caliper Radius from all data: 0.1274663032446158

### Filter Control and treatment groups by PS stratifications

In [ ]:
#Separate groups to chunk by PS distributions (Controls are much larger and need randomization)

filter_CUA = cua_non_logit.filter(cua_non_logit['CUA_ANY'] == 1)  # Filter treatment group
filter_non=cua_non_logit.filter(cua_non_logit['CUA_ANY'] == 0)  # Filter control group

In [ ]:
CUA_count=filter_CUA.count()

In [ ]:
non_count=filter_non.count()
non_count

In [ ]:
highest_prob_cua=filter_CUA.filter(col('prob') > 0.79)
highest_cua_count=highest_prob_cua.count()
print("percent of highest prob CUA:"); (highest_cua_count/CUA_count *100)

#percent of highest prob CUA: 10.596584920244537

print(highest_cua_count)
##3016

In [ ]:
highest_prob_non = filter_non.filter(col('prob') > 0.79)
highest_non_count=highest_prob_non.count()
print("percent of highest prob non:"); (highest_non_count/non_count *100)

#percent of highest prob non:  0.6931692983749175

In [ ]:
concat_highest_prob=highest_prob_non.union(highest_prob_cua)
concat_highest_pdf=concat_highest_prob.toPandas()
print(concat_highest_pdf.count())
print(concat_highest_pdf.head())

In [ ]:
high_prob_cua = filter_CUA.filter((col('prob') > 0.74) & (col('prob') < 0.79))
high_prob_cua_count=high_prob_cua.count()
print("percent of high prob CUA:"); (high_prob_cua_count/CUA_count *100)

#percent of high prob CUA: 18.09078771695594

In [ ]:
high_prob_non = filter_non.filter((col('prob') > 0.74) & (col('prob') < 0.79))
high_non_count=high_prob_non.count()
print("percent of high prob NON:"); (high_non_count/non_count *100)

#percent of high prob NON: 2.7502071307939544

In [ ]:
concat_high_prob=high_prob_non.union(high_prob_cua)
concat_high_pdf=concat_high_prob.toPandas()
print(concat_high_pdf.count())
print(concat_high_pdf.head())

In [ ]:
mid_prob_cua = filter_CUA.filter((col('prob') > 0.68) & (col('prob') < 0.74))
mid_prob_cua_count=mid_prob_cua.count()
print("percent of mid prob CUA:"); (mid_prob_cua_count/CUA_count *100)

#percent of mid prob CUA: 19.27130911390626

In [ ]:
mid_prob_non = filter_non.filter((col('prob') > 0.68) & (col('prob') < 0.74))
mid_prob_non_count=mid_prob_non.count()
print("percent of mid prob non:"); (mid_prob_non_count/non_count *100)

#percent of mid prob non: 3.8557980680282933


In [ ]:
##control population so large that it require random sample, or will not run in pandas

random_mid_prob_sample = mid_prob_non.sample(fraction=200000 / mid_prob_non.count(), withReplacement=False)
concat_mid_prob=random_mid_prob_sample.union(mid_prob_cua)
concat_mid_pdf=concat_mid_prob.toPandas()
print(concat_mid_pdf.count())
print(concat_mid_pdf.head())

In [ ]:
mid_prob_cua2 = filter_CUA.filter((col('prob') > 0.51) & (col('prob') < 0.68))
mid_prob_cua_count2=mid_prob_cua2.count()
print("percent of mid prob CUA2:"); (mid_prob_cua_count2/CUA_count *100)

#percent of mid prob CUA2: 18.298081652729955

In [ ]:
mid_prob_non2 = filter_non.filter((col('prob') > 0.51) & (col('prob') < 0.68))
mid_prob_non_count2=mid_prob_non2.count()
print("percent of mid prob non:"); (mid_prob_non_count2/non_count *100)

#percent of mid prob non: 5.5574400797277645

In [ ]:
random_mid_prob_sample2 = mid_prob_non2.sample(fraction=200000 / mid_prob_non2.count(), withReplacement=False)
concat_mid_prob2=random_mid_prob_sample2.union(mid_prob_cua2)
concat_mid_pdf2=concat_mid_prob2.toPandas()
print(concat_mid_pdf2.count())
print(concat_mid_pdf2.head())

In [ ]:
low_prob_cua = filter_CUA.filter((col('prob1') > 0.46) & (col('prob1') < 0.51))
low_cua_count=low_prob_cua.count()
print("percent of low prob CUA:"); (low_cua_count/CUA_count *100)

#percent of low prob CUA: 18.55807743658211

In [ ]:
low_prob_non = filter_non.filter((col('prob1') > 0.46) & (col('prob1') < 0.51))
low_non_count=low_prob_non.count()
print("percent of low prob NON:"); (low_non_count/non_count *100)

#percent of low prob NON: 17.44869052128403

In [ ]:
random_low_prob_sample = low_prob_non.sample(fraction=250000 / low_prob_non.count(), withReplacement=False)
concat_low_prob=random_low_prob_sample.union(low_prob_cua)
concat_low_pdf=concat_low_prob.toPandas()
print(concat_low_pdf.count())
print(concat_low_pdf.head())

In [ ]:
low_prob_cua2 = filter_CUA.filter((col('prob1') < 0.46))
low_cua_count2=low_prob_cua2.count()
print("percent of low prob CUA:"); (low_cua_count2/CUA_count *100)

#percent of low prob CUA: 15.185159159581197

In [ ]:
low_prob_non2 = filter_non.filter((col('prob1') < 0.46))
low_non_count2=low_prob_non2.count()
print("percent of low prob NON:"); (low_non_count2/non_count *100)

#percent of low prob NON:
69.69469490179104

In [ ]:
random_low_prob_sample2 = low_prob_non2.sample(fraction=200000 / low_prob_non2.count(), withReplacement=False)
concat_low_prob2=random_low_prob_sample2.union(low_prob_cua2)
concat_low_pdf2=concat_low_prob2.toPandas()
print(concat_low_pdf2.count())
print(concat_low_pdf2.head())

## PS Matching with KNN 

### Match on Low Group

In [ ]:
##Set 1

# Separate data by treatment and control
control_data = concat_low_pdf[concat_low_pdf['CUA_ANY'] == 0]['prob']
treatment_data = concat_low_pdf[concat_low_pdf['CUA_ANY'] == 1]['prob']

# Create histograms for treatment and control groups
plt.hist(control_data, alpha=0.5, label='Control')
plt.hist(treatment_data, alpha=0.5, label='Treatment')

plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
# Assuming propensity scores are stored in the 'prob' column 
# Assuming 'cua_any' column indicates treatment group (1 for treatment, 0 for control)
treatment_df = concat_low_pdf[concat_low_pdf['CUA_ANY'] == 1].copy()
control_df = concat_low_pdf[concat_low_pdf['CUA_ANY'] == 0].copy()

# Fit Nearest Neighbors classifier for control group
n_neighbors = 1
knn = NearestNeighbors(n_neighbors=n_neighbors)
knn.fit(control_df[['prob']])  # Fit on control propensity scores

# Find nearest neighbors for treatment group
distances, indices = knn.kneighbors(treatment_df[['prob']])

# Keep track of used control IDs
used_control_ids = set()

# Initialize lists to store matched indices
matched_control_indices = []
matched_treatment_indices = []

# Iterate through treatment samples and find nearest neighbor in control group
for i, neighbors in enumerate(indices):
    nearest_control_index = neighbors[0]  # Retrieve the index of the nearest neighbor
    control_id = control_df.iloc[nearest_control_index]['personid']
    if control_id not in used_control_ids:
        matched_control_indices.append(nearest_control_index)
        matched_treatment_indices.append(i)
        used_control_ids.add(control_id)  # Mark control ID as used

# Create a DataFrame containing treatment and matched control individuals
matched_df = pd.DataFrame({
    'Treatment_PersonID': treatment_df.iloc[matched_treatment_indices]['personid'].values,
    'Treatment_Propensity_Score': treatment_df.iloc[matched_treatment_indices]['prob'].values,
    'Control_PersonID': control_df.iloc[matched_control_indices]['personid'].values,
    'Control_Propensity_Score': control_df.iloc[matched_control_indices]['prob'].values
})

# Print the matched DataFrame
print(matched_df)


In [ ]:
# Assuming propensity scores are stored in the 'prob' column 
# Assuming 'cua_any' column indicates treatment group (1 for treatment, 0 for control)
treatment_df = concat_low_pdf[concat_low_pdf['CUA_ANY'] == 1].copy()
control_df = concat_low_pdf[concat_low_pdf['CUA_ANY'] == 0].copy()

# Fit Nearest Neighbors classifier for control group
n_neighbors = len(control_df)  # Set number of neighbors to the size of control group
knn = NearestNeighbors(n_neighbors=n_neighbors)
knn.fit(control_df[['prob']])  # Fit on control propensity scores

# Find nearest neighbors for treatment group
distances, indices = knn.kneighbors(treatment_df[['prob']])

# Keep track of used control IDs
used_control_ids = set()

# Initialize lists to store matched indices
matched_control_indices = []
matched_treatment_indices = []

# Iterate through treatment samples and find nearest neighbor in control group
for i, neighbors in enumerate(indices):
    for nearest_control_index in neighbors:  # Iterate over neighbors
        control_id = control_df.iloc[nearest_control_index]['personid']
        if control_id not in used_control_ids:
            matched_control_indices.append(nearest_control_index)
            matched_treatment_indices.append(i)
            used_control_ids.add(control_id)  # Mark control ID as used
            break  # Move to the next treatment individual

# Create a DataFrame containing treatment and matched control individuals
matched_df = pd.DataFrame({
    'Treatment_PersonID': treatment_df.iloc[matched_treatment_indices]['personid'].values,
    'Treatment_Propensity_Score': treatment_df.iloc[matched_treatment_indices]['prob'].values,
    'Control_PersonID': control_df.iloc[matched_control_indices]['personid'].values,
    'Control_Propensity_Score': control_df.iloc[matched_control_indices]['prob'].values
})

# Print the matched DataFrame
print(matched_df)



In [ ]:
matched_df.head()

In [ ]:
matched_df.count()
#5282

In [ ]:
# Check for missing values (NAs) in the 'Control_PersonID' column
missing_values = matched_df['Control_PersonID'].isna().sum()
print("Number of missing values in 'Control_PersonID' column:", missing_values)

# Check for duplicates in the 'Control_PersonID' column
duplicate_count = matched_df.duplicated(subset=['Control_PersonID']).sum()
print("Number of duplicate IDs in 'Control_PersonID' column:", duplicate_count)

#Number of missing values in 'Control_PersonID' column: 0
#Number of duplicate IDs in 'Control_PersonID' column: 0

In [ ]:
##Show the After histogram

# Filter propensity scores for treatment and control groups
treatment_scores = matched_df['Treatment_Propensity_Score']
control_scores = matched_df['Control_Propensity_Score']

# Plot histograms
##Adjust alpha
plt.hist(treatment_scores, bins=20, alpha=0.3, label='Treatment')
plt.hist(control_scores, bins=20, alpha=0.3, label='Control')

# Add labels and title
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.title('Histogram of Matched Propensity Scores by Treatment vs. Control')

# Add legend
plt.legend()

# Show plot
plt.show()

In [ ]:
# Calculate the absolute difference in propensity scores between treatment and control
matched_df['Propensity_Score_Difference'] = abs(matched_df['Treatment_Propensity_Score'] - matched_df['Control_Propensity_Score'])

# Maximum difference
max_difference = matched_df['Propensity_Score_Difference'].max()

# Average difference
avg_difference = matched_df['Propensity_Score_Difference'].mean()

print("Maximum Difference in Propensity Scores:", max_difference)
print("Average Difference in Propensity Scores:", avg_difference)

#Maximum Difference in Propensity Scores: 9.384839348480778e-05
#Average Difference in Propensity Scores: 2.2879055039834536e-07

### Match on Low2 Group

In [ ]:
##Set 2

# Separate data by treatment and control
control_data = concat_low_pdf2[concat_low_pdf2['CUA_ANY'] == 0]['prob']
treatment_data = concat_low_pdf2[concat_low_pdf2['CUA_ANY'] == 1]['prob']

# Create histograms for treatment and control groups
plt.hist(control_data, alpha=0.5, label='Control')
plt.hist(treatment_data, alpha=0.5, label='Treatment')

plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:

# Assuming propensity scores are stored in the 'prob' column of concat_low_pdf
# Assuming 'cua_any' column indicates treatment group (1 for treatment, 0 for control)
treatment_df2 = concat_low_pdf2[concat_low_pdf2['CUA_ANY'] == 1].copy()
control_df2 = concat_low_pdf2[concat_low_pdf2['CUA_ANY'] == 0].copy()

# Fit Nearest Neighbors classifier for control group
n_neighbors2 = len(control_df2)  # Set number of neighbors to the size of control group
knn2 = NearestNeighbors(n_neighbors=n_neighbors2)
knn2.fit(control_df2[['prob']])  # Fit on control propensity scores

# Find nearest neighbors for treatment group
distances, indices = knn2.kneighbors(treatment_df2[['prob']])

# Keep track of used control IDs
used_control_ids2 = set()

# Initialize lists to store matched indices
matched_control_indices2 = []
matched_treatment_indices2 = []

# Iterate through treatment samples and find nearest neighbor in control group
for i, neighbors in enumerate(indices):
    for nearest_control_index2 in neighbors:  # Iterate over neighbors
        control_id2 = control_df2.iloc[nearest_control_index2]['personid']
        if control_id2 not in used_control_ids2:
            matched_control_indices2.append(nearest_control_index2)
            matched_treatment_indices2.append(i)
            used_control_ids2.add(control_id2)  # Mark control ID as used
            break  # Move to the next treatment individual

# Create a DataFrame containing treatment and matched control individuals
matched_df2 = pd.DataFrame({
    'Treatment_PersonID': treatment_df2.iloc[matched_treatment_indices2]['personid'].values,
    'Treatment_Propensity_Score': treatment_df2.iloc[matched_treatment_indices2]['prob'].values,
    'Control_PersonID': control_df2.iloc[matched_control_indices2]['personid'].values,
    'Control_Propensity_Score': control_df2.iloc[matched_control_indices2]['prob'].values
})

# Print the matched DataFrame
print(matched_df2)

In [ ]:
matched_df2.count()
##4322

In [ ]:
# Check for missing values (NAs) in the 'Control_PersonID' column
missing_values = matched_df2['Control_PersonID'].isna().sum()
print("Number of missing values in 'Control_PersonID' column:", missing_values)

# Check for duplicates in the 'Control_PersonID' column
duplicate_count = matched_df2.duplicated(subset=['Control_PersonID']).sum()
print("Number of duplicate IDs in 'Control_PersonID' column:", duplicate_count)

#Number of missing values in 'Control_PersonID' column: 0
#Number of duplicate IDs in 'Control_PersonID' column: 0

In [ ]:
# Filter propensity scores for treatment and control groups
treatment_scores = matched_df2['Treatment_Propensity_Score']
control_scores = matched_df2['Control_Propensity_Score']

# Plot histograms
plt.hist(treatment_scores, bins=20, alpha=0.3, label='Treatment')
plt.hist(control_scores, bins=20, alpha=0.3, label='Control')

# Add labels and title
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.title('Histogram of Matched Propensity Scores by Treatment vs. Control')

# Add legend
plt.legend()

# Show plot
plt.show()

In [ ]:
# Calculate the absolute difference in propensity scores between treatment and control
matched_df2['Propensity_Score_Difference'] = abs(matched_df2['Treatment_Propensity_Score'] - matched_df2['Control_Propensity_Score'])

# Maximum difference
max_difference = matched_df2['Propensity_Score_Difference'].max()

# Average difference
avg_difference = matched_df2['Propensity_Score_Difference'].mean()

print("Maximum Difference in Propensity Scores:", max_difference)
print("Average Difference in Propensity Scores:", avg_difference)

#Maximum Difference in Propensity Scores: 0.00016796650844513872
#Average Difference in Propensity Scores: 1.7268268506190105e-06

### Concatenate Low & Low2

In [ ]:
concatenated_low_match = pd.concat([matched_df, matched_df2], ignore_index=True)
concatenated_low_match.head()

In [ ]:
print(len(concatenated_low_match))
print(low_cua_count + low_cua_count2)
#9604
#9604

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
spark_low_match = spark.createDataFrame(concatenated_low_match)

### Match on Mid

In [ ]:
# Separate data by treatment and control
control_data = concat_mid_pdf[concat_mid_pdf['CUA_ANY'] == 0]['prob']
treatment_data = concat_mid_pdf[concat_mid_pdf['CUA_ANY'] == 1]['prob']

# Create histograms for treatment and control groups
plt.hist(control_data, alpha=0.5, label='Control')
plt.hist(treatment_data, alpha=0.5, label='Treatment')

plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
# Assuming propensity scores are stored in the 'prob' column of concat_low_pdf
# Assuming 'cua_any' column indicates treatment group (1 for treatment, 0 for control)
treatment_df3 = concat_mid_pdf[concat_mid_pdf['CUA_ANY'] == 1].copy()
control_df3 = concat_mid_pdf[concat_mid_pdf['CUA_ANY'] == 0].copy()

# Fit Nearest Neighbors classifier for control group
n_neighbors3 = len(control_df3)  # Set number of neighbors to the size of control group
knn3 = NearestNeighbors(n_neighbors=n_neighbors3)
knn3.fit(control_df3[['prob']])  # Fit on control propensity scores

# Find nearest neighbors for treatment group
distances, indices = knn3.kneighbors(treatment_df3[['prob']])

# Keep track of used control IDs
used_control_ids3 = set()

# Initialize lists to store matched indices
matched_control_indices3 = []
matched_treatment_indices3 = []

# Iterate through treatment samples and find nearest neighbor in control group
for i, neighbors in enumerate(indices):
    for nearest_control_index3 in neighbors:  # Iterate over neighbors
        control_id3 = control_df3.iloc[nearest_control_index3]['personid']
        if control_id3 not in used_control_ids3:
            matched_control_indices3.append(nearest_control_index3)
            matched_treatment_indices3.append(i)
            used_control_ids3.add(control_id3)  # Mark control ID as used
            break  # Move to the next treatment individual

# Create a DataFrame containing treatment and matched control individuals
matched_df3 = pd.DataFrame({
    'Treatment_PersonID': treatment_df3.iloc[matched_treatment_indices3]['personid'].values,
    'Treatment_Propensity_Score': treatment_df3.iloc[matched_treatment_indices3]['prob'].values,
    'Control_PersonID': control_df3.iloc[matched_control_indices3]['personid'].values,
    'Control_Propensity_Score': control_df3.iloc[matched_control_indices3]['prob'].values
})

# Print the matched DataFrame
print(matched_df3)

In [ ]:
# Check for missing values (NAs) in the 'Control_PersonID' column
missing_values = matched_df3['Control_PersonID'].isna().sum()
print("Number of missing values in 'Control_PersonID' column:", missing_values)

# Check for duplicates in the 'Control_PersonID' column
duplicate_count = matched_df3.duplicated(subset=['Control_PersonID']).sum()
print("Number of duplicate IDs in 'Control_PersonID' column:", duplicate_count)

#Number of missing values in 'Control_PersonID' column: 0
#Number of duplicate IDs in 'Control_PersonID' column: 0

In [ ]:
# Filter propensity scores for treatment and control groups
treatment_scores = matched_df3['Treatment_Propensity_Score']
control_scores = matched_df3['Control_Propensity_Score']

# Plot histograms
plt.hist(treatment_scores, bins=20, alpha=0.3, label='Treatment')
plt.hist(control_scores, bins=20, alpha=0.3, label='Control')

# Add labels and title
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.title('Histogram of Matched Propensity Scores by Treatment vs. Control')

# Add legend
plt.legend()

# Show plot
plt.show()

In [ ]:
# Calculate the absolute difference in propensity scores between treatment and control
matched_df3['Propensity_Score_Difference'] = abs(matched_df3['Treatment_Propensity_Score'] - matched_df3['Control_Propensity_Score'])

# Maximum difference
max_difference = matched_df3['Propensity_Score_Difference'].max()

# Average difference
avg_difference = matched_df3['Propensity_Score_Difference'].mean()

print("Maximum Difference in Propensity Scores:", max_difference)
print("Average Difference in Propensity Scores:", avg_difference)

#Maximum Difference in Propensity Scores: 1.3726183205164944e-05
#Average Difference in Propensity Scores: 1.6642898049665893e-07

In [ ]:
# Separate data by treatment and control
control_data = concat_mid_pdf2[concat_mid_pdf2['CUA_ANY'] == 0]['prob']
treatment_data = concat_mid_pdf2[concat_mid_pdf2['CUA_ANY'] == 1]['prob']

# Create histograms for treatment and control groups
plt.hist(control_data, alpha=0.5, label='Control')
plt.hist(treatment_data, alpha=0.5, label='Treatment')

plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()

### Match for Mid2

In [ ]:
# Assuming propensity scores are stored in the 'prob' column 
# Assuming 'cua_any' column indicates treatment group (1 for treatment, 0 for control)
treatment_df4 = concat_mid_pdf2[concat_mid_pdf2['CUA_ANY'] == 1].copy()
control_df4 = concat_mid_pdf2[concat_mid_pdf2['CUA_ANY'] == 0].copy()

# Fit Nearest Neighbors classifier for control group
n_neighbors4 = len(control_df4)  # Set number of neighbors to the size of control group
knn4 = NearestNeighbors(n_neighbors=n_neighbors4)
knn4.fit(control_df4[['prob']])  # Fit on control propensity scores

# Find nearest neighbors for treatment group
distances, indices = knn4.kneighbors(treatment_df4[['prob']])

# Keep track of used control IDs
used_control_ids4 = set()

# Initialize lists to store matched indices
matched_control_indices4 = []
matched_treatment_indices4 = []

# Iterate through treatment samples and find nearest neighbor in control group
for i, neighbors in enumerate(indices):
    for nearest_control_index4 in neighbors:  # Iterate over neighbors
        control_id4 = control_df4.iloc[nearest_control_index4]['personid']
        if control_id4 not in used_control_ids4:
            matched_control_indices4.append(nearest_control_index4)
            matched_treatment_indices4.append(i)
            used_control_ids4.add(control_id4)  # Mark control ID as used
            break  # Move to the next treatment individual

# Create a DataFrame containing treatment and matched control individuals
matched_df4 = pd.DataFrame({
    'Treatment_PersonID': treatment_df4.iloc[matched_treatment_indices4]['personid'].values,
    'Treatment_Propensity_Score': treatment_df4.iloc[matched_treatment_indices4]['prob'].values,
    'Control_PersonID': control_df4.iloc[matched_control_indices4]['personid'].values,
    'Control_Propensity_Score': control_df4.iloc[matched_control_indices4]['prob'].values
})

# Print the matched DataFrame
print(matched_df4)


In [ ]:
matched_df4.count()
#5208

In [ ]:
# Check for missing values (NAs) in the 'Control_PersonID' column
missing_values = matched_df4['Control_PersonID'].isna().sum()
print("Number of missing values in 'Control_PersonID' column:", missing_values)

# Check for duplicates in the 'Control_PersonID' column
duplicate_count = matched_df4.duplicated(subset=['Control_PersonID']).sum()
print("Number of duplicate IDs in 'Control_PersonID' column:", duplicate_count)

#Number of missing values in 'Control_PersonID' column: 0
#Number of duplicate IDs in 'Control_PersonID' column: 0


In [ ]:
# Filter propensity scores for treatment and control groups
treatment_scores = matched_df4['Treatment_Propensity_Score']
control_scores = matched_df4['Control_Propensity_Score']

# Plot histograms
plt.hist(treatment_scores, bins=20, alpha=0.3, label='Treatment')
plt.hist(control_scores, bins=20, alpha=0.3, label='Control')

# Add labels and title
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.title('Histogram of Matched Propensity Scores by Treatment vs. Control')

# Add legend
plt.legend()

# Show plot
plt.show()

In [ ]:
# Calculate the absolute difference in propensity scores between treatment and control
matched_df4['Propensity_Score_Difference'] = abs(matched_df4['Treatment_Propensity_Score'] - matched_df4['Control_Propensity_Score'])

# Maximum difference
max_difference = matched_df4['Propensity_Score_Difference'].max()

# Average difference
avg_difference = matched_df4['Propensity_Score_Difference'].mean()

print("Maximum Difference in Propensity Scores:", max_difference)
print("Average Difference in Propensity Scores:", avg_difference)

#Maximum Difference in Propensity Scores: 2.966165954598754e-05
#Average Difference in Propensity Scores: 6.579066603665349e-07

### Concatenate Mid & Mid2

In [ ]:
concatenated_mid_match = pd.concat([matched_df3, matched_df4], ignore_index=True)
print(len(concatenated_mid_match))
print(mid_prob_cua_count + mid_prob_cua_count2)

#10693
#10693

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
spark_mid_match = spark.createDataFrame(concatenated_mid_match)

### Match on High

In [ ]:
# Separate data by treatment and control
control_data = concat_highest_pdf[concat_highest_pdf['CUA_ANY'] == 0]['prob']
treatment_data = concat_highest_pdf[concat_highest_pdf['CUA_ANY'] == 1]['prob']

# Create histograms for treatment and control groups
plt.hist(control_data, alpha=0.5, label='Control')
plt.hist(treatment_data, alpha=0.5, label='Treatment')

plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.legend()

In [ ]:
# Assuming propensity scores are stored in the 'prob' column 
# Assuming 'cua_any' column indicates treatment group (1 for treatment, 0 for control)
treatment_df5 = concat_highest_pdf[concat_highest_pdf['CUA_ANY'] == 1].copy()
control_df5 = concat_highest_pdf[concat_highest_pdf['CUA_ANY'] == 0].copy()

# Fit Nearest Neighbors classifier for control group
n_neighbors5 = len(control_df5)  # Set number of neighbors to the size of control group
knn5 = NearestNeighbors(n_neighbors=n_neighbors5)
knn5.fit(control_df5[['prob']])  # Fit on control propensity scores

# Find nearest neighbors for treatment group
distances, indices = knn5.kneighbors(treatment_df5[['prob']])

# Keep track of used control IDs
used_control_ids5 = set()

# Initialize lists to store matched indices
matched_control_indices5 = []
matched_treatment_indices5 = []

# Iterate through treatment samples and find nearest neighbor in control group
for i, neighbors in enumerate(indices):
    for nearest_control_index5 in neighbors:  # Iterate over neighbors
        control_id5 = control_df5.iloc[nearest_control_index5]['personid']
        if control_id5 not in used_control_ids5:
            matched_control_indices5.append(nearest_control_index5)
            matched_treatment_indices5.append(i)
            used_control_ids5.add(control_id5)  # Mark control ID as used
            break  # Move to the next treatment individual

# Create a DataFrame containing treatment and matched control individuals
matched_df5 = pd.DataFrame({
    'Treatment_PersonID': treatment_df5.iloc[matched_treatment_indices5]['personid'].values,
    'Treatment_Propensity_Score': treatment_df5.iloc[matched_treatment_indices5]['prob'].values,
    'Control_PersonID': control_df5.iloc[matched_control_indices5]['personid'].values,
    'Control_Propensity_Score': control_df5.iloc[matched_control_indices5]['prob'].values
})

# Print the matched DataFrame
print(matched_df5)


In [ ]:
matched_df5.count()

#3016

In [ ]:
# Filter propensity scores for treatment and control groups
treatment_scores = matched_df5['Treatment_Propensity_Score']
control_scores = matched_df5['Control_Propensity_Score']

# Plot histograms
plt.hist(treatment_scores, bins=20, alpha=0.3, label='Treatment')
plt.hist(control_scores, bins=20, alpha=0.3, label='Control')

# Add labels and title
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.title('Histogram of Matched Propensity Scores by Treatment vs. Control')

# Add legend
plt.legend()

# Show plot
plt.show()

In [ ]:
# Calculate the absolute difference in propensity scores between treatment and control
matched_df5['Propensity_Score_Difference'] = abs(matched_df5['Treatment_Propensity_Score'] - matched_df5['Control_Propensity_Score'])

# Maximum difference
max_difference = matched_df5['Propensity_Score_Difference'].max()

# Average difference
avg_difference = matched_df5['Propensity_Score_Difference'].mean()

print("Maximum Difference in Propensity Scores:", max_difference)
print("Average Difference in Propensity Scores:", avg_difference)

#Maximum Difference in Propensity Scores: 0.0002882994803843353
#Average Difference in Propensity Scores: 6.399843397289812e-07


### Match on Highest

In [ ]:
# Separate data by treatment and control
control_data = concat_high_pdf[concat_high_pdf['CUA_ANY'] == 0]['prob']
treatment_data = concat_high_pdf[concat_high_pdf['CUA_ANY'] == 1]['prob']

# Create histograms for treatment and control groups
plt.hist(control_data, alpha=0.5, label='Control')
plt.hist(treatment_data, alpha=0.5, label='Treatment')

plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.legend()

In [ ]:
# Assuming propensity scores are stored in the 'prob' column 
# Assuming 'cua_any' column indicates treatment group (1 for treatment, 0 for control)
treatment_df6 = concat_high_pdf[concat_high_pdf['CUA_ANY'] == 1].copy()
control_df6 = concat_high_pdf[concat_high_pdf['CUA_ANY'] == 0].copy()

# Fit Nearest Neighbors classifier for control group
n_neighbors6 = len(control_df6)  # Set number of neighbors to the size of control group
knn6 = NearestNeighbors(n_neighbors=n_neighbors6)
knn6.fit(control_df6[['prob']])  # Fit on control propensity scores

# Find nearest neighbors for treatment group
distances, indices = knn6.kneighbors(treatment_df6[['prob']])

# Keep track of used control IDs
used_control_ids6 = set()

# Initialize lists to store matched indices
matched_control_indices6 = []
matched_treatment_indices6 = []

# Iterate through treatment samples and find nearest neighbor in control group
for i, neighbors in enumerate(indices):
    for nearest_control_index6 in neighbors:  # Iterate over neighbors
        control_id6 = control_df6.iloc[nearest_control_index6]['personid']
        if control_id6 not in used_control_ids6:
            matched_control_indices6.append(nearest_control_index6)
            matched_treatment_indices6.append(i)
            used_control_ids6.add(control_id6)  # Mark control ID as used
            break  # Move to the next treatment individual

# Create a DataFrame containing treatment and matched control individuals
matched_df6 = pd.DataFrame({
    'Treatment_PersonID': treatment_df6.iloc[matched_treatment_indices6]['personid'].values,
    'Treatment_Propensity_Score': treatment_df6.iloc[matched_treatment_indices6]['prob'].values,
    'Control_PersonID': control_df6.iloc[matched_control_indices6]['personid'].values,
    'Control_Propensity_Score': control_df6.iloc[matched_control_indices6]['prob'].values
})

# Print the matched DataFrame
print(matched_df6)

In [ ]:
matched_df6.count()
## 5149

In [ ]:
# Filter propensity scores for treatment and control groups
treatment_scores = matched_df6['Treatment_Propensity_Score']
control_scores = matched_df6['Control_Propensity_Score']

# Plot histograms
plt.hist(treatment_scores, bins=20, alpha=0.3, label='Treatment')
plt.hist(control_scores, bins=20, alpha=0.3, label='Control')

# Add labels and title
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.title('Histogram of Matched Propensity Scores by Treatment vs. Control')

# Add legend
plt.legend()

# Show plot
plt.show()

In [ ]:
# Calculate the absolute difference in propensity scores between treatment and control
matched_df6['Propensity_Score_Difference'] = abs(matched_df6['Treatment_Propensity_Score'] - matched_df6['Control_Propensity_Score'])

# Maximum difference
max_difference = matched_df6['Propensity_Score_Difference'].max()

# Average difference
avg_difference = matched_df6['Propensity_Score_Difference'].mean()

print("Maximum Difference in Propensity Scores:", max_difference)
print("Average Difference in Propensity Scores:", avg_difference)

#Maximum Difference in Propensity Scores: 3.6452751333015954e-06
#Average Difference in Propensity Scores: 4.993818770897132e-08

### Concatenate High & Highest Groups

In [ ]:
concatenated_high_match = pd.concat([matched_df5, matched_df6], ignore_index=True)
print(len(concatenated_high_match))
print(high_prob_cua_count + highest_cua_count)

#8165
#8165

In [ ]:
spark_high_match = spark.createDataFrame(concatenated_high_match)

### Concatenate the various matches DFs to save as one table

In [ ]:
high_mid=spark_high_match.union(spark_mid_match)

In [ ]:
high_low=high_mid.union(spark_low_match)

In [ ]:
high_low.limit(10).toPandas()

In [ ]:
high_low.write.saveAsTable('CUA_db.ps_matches_updated')